# Experiment 9: Random Forest + Reject Option Classification (Post-Processing)
##### Full pipeline: retrain on LendingClub, post-process fairness, validation (GC & GMSC), metrics before/after, SHAP, and saving results

In [ ]:


# Step 0: Setup
!pip install aif360 shap scikit-learn pandas matplotlib seaborn --quiet

import os
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric
from aif360.algorithms.postprocessing import RejectOptionClassification

RESULTS_DIR = '/content/drive/MyDrive/Research_Thesis_Implementation/Validation files & results/Validation Results _germanCredit &GivemesomeCredit'
os.makedirs(RESULTS_DIR, exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 5.8 MB/s eta 0:00:00


pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [ ]:


# -----------------------------
# Step 1: Load & preprocess LendingClub
# -----------------------------
DATA_PATH = '/content/drive/MyDrive/Research_Thesis_Implementation/data_final/lendingclub_data.csv'
df = pd.read_csv(DATA_PATH)

selected_cols = ['loan_status', 'annual_inc', 'term', 'grade', 'home_ownership', 'purpose', 'zip_code']
df = df[selected_cols].dropna().copy()

df['loan_status'] = df['loan_status'].apply(lambda x: 1 if x == 'Fully Paid' else 0)

for col in ['term', 'grade', 'home_ownership', 'purpose', 'zip_code']:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

df['annual_inc'] = StandardScaler().fit_transform(df[['annual_inc']])

privileged_groups = [{'zip_code': 1}]
unprivileged_groups = [{'zip_code': 0}]

aif_data = StandardDataset(
    df,
    label_name='loan_status',
    favorable_classes=[1],
    protected_attribute_names=['zip_code'],
    privileged_classes=[[1]]
)

/tmp/ipython-input-3486541711.py:5: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH)


In [ ]:


# -----------------------------
# Step 2: Train RandomForest and apply ROC (post-processing)
# -----------------------------
X = aif_data.features
y = aif_data.labels.ravel()

rf = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42)
rf.fit(X, y)
y_pred = rf.predict(X)
y_prob = rf.predict_proba(X)[:, 1]

pred_dataset = aif_data.copy()
pred_dataset.labels = y_pred.reshape(-1, 1)
pred_dataset.scores = y_prob.reshape(-1, 1)

roc = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name="Statistical parity difference",
    metric_ub=0.05,
    metric_lb=-0.05
)
roc = roc.fit(aif_data, pred_dataset)
post_dataset = roc.predict(pred_dataset)

In [ ]:


# -----------------------------
# Step 3: Metrics & fairness on LendingClub (before vs after ROC)
# -----------------------------
def perf_metrics(y_true, y_pred, y_prob):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'AUC': roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan
    }

pre_metrics = perf_metrics(y, y_pred, y_prob)
post_metrics = perf_metrics(y, post_dataset.labels.ravel(), post_dataset.scores.ravel())

metric_pre = ClassificationMetric(aif_data, pred_dataset,
                                  unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups)
metric_post = ClassificationMetric(aif_data, post_dataset,
                                   unprivileged_groups=unprivileged_groups,
                                   privileged_groups=privileged_groups)

fairness_pre = {
    'SPD': metric_pre.statistical_parity_difference(),
    'DI': metric_pre.disparate_impact(),
    'EOD': metric_pre.equal_opportunity_difference(),
    'AOD': metric_pre.average_odds_difference(),
    'BiasAmp': metric_pre.between_group_generalized_entropy_index(),
    'Theil': metric_pre.theil_index()
}

fairness_post = {
    'SPD': metric_post.statistical_parity_difference(),
    'DI': metric_post.disparate_impact(),
    'EOD': metric_post.equal_opportunity_difference(),
    'AOD': metric_post.average_odds_difference(),
    'BiasAmp': metric_post.between_group_generalized_entropy_index(),
    'Theil': metric_post.theil_index()
}

print("=== LendingClub (RandomForest + ROC) ===")
print("Performance BEFORE ROC:", pre_metrics)
print("Performance AFTER  ROC:", post_metrics)
print("Fairness BEFORE ROC:", fairness_pre)
print("Fairness AFTER  ROC:", fairness_post)

=== LendingClub (RandomForest + ROC) ===
Performance BEFORE ROC: {'Accuracy': 0.9994, 'Precision': 0.999450247388675, 'Recall': 0.9997250481165796, 'F1': 0.9995876288659794, 'AUC': np.float64(0.9999990922351611)}
Performance AFTER  ROC: {'Accuracy': 0.8066, 'Precision': 1.0, 'Recall': 0.7341215287324718, 'F1': 0.8466782939590931, 'AUC': np.float64(0.9999990922351611)}
Fairness BEFORE ROC: {'SPD': np.float64(0.42307692307692313), 'DI': np.float64(1.8461538461538463), 'EOD': np.float64(0.0), 'AOD': np.float64(0.0), 'BiasAmp': np.float64(166.16666666666666), 'Theil': np.float64(0.0003544268604090961)}
Fairness AFTER  ROC: {'SPD': np.float64(0.038461538461538436), 'DI': np.float64(1.0769230769230769), 'EOD': np.float64(-0.41666666666666663), 'AOD': np.float64(-0.20833333333333331), 'BiasAmp': np.float64(172.57692307692307), 'Theil': np.float64(0.21492739654286933)}


In [ ]:


# -----------------------------
# Step 4: SHAP (TreeExplainer) for LendingClub (global + subgroup)
# -----------------------------
X_df = pd.DataFrame(X, columns=aif_data.feature_names)
explainer = shap.TreeExplainer(rf)
raw_shap = explainer.shap_values(X_df)

# Binary classifier SHAP returns list [class0, class1]; use class 1
shap_arr = raw_shap[1] if isinstance(raw_shap, list) and len(raw_shap) > 1 else raw_shap

plt.figure(figsize=(10, 6))
shap.summary_plot(shap_arr, X_df, show=False)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/exp9_lc_shap_global.png', dpi=150, bbox_inches='tight')
plt.close()

priv_mask = X_df['zip_code'] == 1
unpriv_mask = X_df['zip_code'] == 0

if priv_mask.sum() > 0:
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_arr[priv_mask], X_df[priv_mask], show=False)
    plt.title('SHAP Summary — Privileged (zip_code=1)')
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/exp9_lc_shap_privileged.png', dpi=150, bbox_inches='tight')
    plt.close()

if unpriv_mask.sum() > 0:
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_arr[unpriv_mask], X_df[unpriv_mask], show=False)
    plt.title('SHAP Summary — Unprivileged (zip_code=0)')
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/exp9_lc_shap_unprivileged.png', dpi=150, bbox_inches='tight')
    plt.close()

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [ ]:


# -----------------------------
# Step 5: Validation on GermanCredit (schema-aligned) + post-processing
# -----------------------------
GC_PATH = '/content/drive/MyDrive/Research_Thesis_Implementation/Validation files & results/Validation dataset/german_credit_data.csv'
df_gc = pd.read_csv(GC_PATH)

# Target mapping
if 'Risk' in df_gc.columns:
    df_gc['loan_status'] = df_gc['Risk'].map({'good': 1, 'bad': 0})
elif 'Creditability' in df_gc.columns:
    df_gc['loan_status'] = df_gc['Creditability']
elif 'class' in df_gc.columns:
    df_gc['loan_status'] = df_gc['class'].map({'good': 1, 'bad': 0})
else:
    raise ValueError("Target column not found in GermanCredit.")

for col in df_gc.columns:
    if df_gc[col].dtype == 'object':
        df_gc[col] = LabelEncoder().fit_transform(df_gc[col].astype(str))

common_gc = pd.DataFrame()
common_gc['annual_inc']     = df_gc['Credit amount']
common_gc['term']           = df_gc['Duration']
common_gc['grade']          = df_gc['Purpose']
common_gc['home_ownership'] = df_gc['Housing']
common_gc['purpose']        = df_gc['Purpose']
common_gc['zip_code']       = df_gc['Checking account']
common_gc['loan_status']    = df_gc['loan_status']

for col in ['term', 'grade', 'home_ownership', 'purpose', 'zip_code']:
    common_gc[col] = LabelEncoder().fit_transform(common_gc[col].astype(str))
common_gc['annual_inc'] = StandardScaler().fit_transform(common_gc[['annual_inc']])

aif_gc = StandardDataset(
    common_gc,
    label_name='loan_status',
    favorable_classes=[1],
    protected_attribute_names=['zip_code'],
    privileged_classes=[[1]]
)

X_gc = aif_gc.features
y_gc = aif_gc.labels.ravel()

y_pred_gc = rf.predict(X_gc)
y_prob_gc = rf.predict_proba(X_gc)[:, 1]

pred_gc = aif_gc.copy()
pred_gc.labels = y_pred_gc.reshape(-1, 1)
pred_gc.scores = y_prob_gc.reshape(-1, 1)

roc_gc = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name="Statistical parity difference",
    metric_ub=0.05,
    metric_lb=-0.05
).fit(aif_gc, pred_gc)

post_gc = roc_gc.predict(pred_gc)

pre_gc_metrics = perf_metrics(y_gc, y_pred_gc, y_prob_gc)
post_gc_metrics = perf_metrics(y_gc, post_gc.labels.ravel(), post_gc.scores.ravel())

metric_gc_pre = ClassificationMetric(aif_gc, pred_gc,
                                     unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups)
metric_gc_post = ClassificationMetric(aif_gc, post_gc,
                                      unprivileged_groups=unprivileged_groups,
                                      privileged_groups=privileged_groups)

fair_gc_pre = {
    'SPD': metric_gc_pre.statistical_parity_difference(),
    'DI': metric_gc_pre.disparate_impact(),
    'EOD': metric_gc_pre.equal_opportunity_difference(),
    'AOD': metric_gc_pre.average_odds_difference(),
    'BiasAmp': metric_gc_pre.between_group_generalized_entropy_index(),
    'Theil': metric_gc_pre.theil_index()
}
fair_gc_post = {
    'SPD': metric_gc_post.statistical_parity_difference(),
    'DI': metric_gc_post.disparate_impact(),
    'EOD': metric_gc_post.equal_opportunity_difference(),
    'AOD': metric_gc_post.average_odds_difference(),
    'BiasAmp': metric_gc_post.between_group_generalized_entropy_index(),
    'Theil': metric_gc_post.theil_index()
}

print("\n=== GermanCredit (RF + ROC) ===")
print("Performance BEFORE ROC:", pre_gc_metrics)
print("Performance AFTER  ROC:", post_gc_metrics)
print("Fairness BEFORE ROC:", fair_gc_pre)
print("Fairness AFTER  ROC:", fair_gc_post)

# SHAP for GC
X_gc_df = pd.DataFrame(X_gc, columns=aif_gc.feature_names)
raw_shap_gc = explainer.shap_values(X_gc_df)
shap_gc = raw_shap_gc[1] if isinstance(raw_shap_gc, list) and len(raw_shap_gc) > 1 else raw_shap_gc

plt.figure(figsize=(10, 6))
shap.summary_plot(shap_gc, X_gc_df, show=False)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/exp9_gc_shap_global.png', dpi=150, bbox_inches='tight')
plt.close()


=== GermanCredit (RF + ROC) ===
Performance BEFORE ROC: {'Accuracy': 0.461, 'Precision': 0.7319884726224783, 'Recall': 0.3628571428571429, 'F1': 0.48519579751671443, 'AUC': np.float64(0.5408952380952381)}
Performance AFTER  ROC: {'Accuracy': 0.4, 'Precision': 0.8048780487804879, 'Recall': 0.18857142857142858, 'F1': 0.3055555555555556, 'AUC': np.float64(0.5408952380952381)}
Fairness BEFORE ROC: {'SPD': np.float64(0.04945323311535016), 'DI': np.float64(1.170550252667041), 'EOD': np.float64(0.08141779259519216), 'AOD': np.float64(0.04970360529230508), 'BiasAmp': np.float64(0.43007101405108883), 'Theil': np.float64(0.6346754073315453)}
Fairness AFTER  ROC: {'SPD': np.float64(0.038110872927577144), 'DI': np.float64(1.341727493917275), 'EOD': np.float64(0.04070889629759608), 'AOD': np.float64(0.047338575132925026), 'BiasAmp': np.float64(0.4346575420981001), 'Theil': np.float64(0.8634772344193222)}


<Figure size 1000x600 with 0 Axes>

In [ ]:


# -----------------------------
# Step 6: Validation on GiveMeSomeCredit (schema-aligned) + post-processing
# -----------------------------
GMSC_PATH = '/content/drive/MyDrive/Research_Thesis_Implementation/Validation files & results/Validation dataset/GiveMeSomeCredit.csv'
df_gmsc = pd.read_csv(GMSC_PATH)

df_gmsc['loan_status'] = 1 - df_gmsc['SeriousDlqin2yrs']

for col in df_gmsc.columns:
    if df_gmsc[col].dtype == 'object':
        df_gmsc[col] = LabelEncoder().fit_transform(df_gmsc[col].astype(str))

common_gmsc = pd.DataFrame()
common_gmsc['annual_inc']     = df_gmsc['MonthlyIncome'].fillna(df_gmsc['MonthlyIncome'].median())
common_gmsc['term']           = df_gmsc['NumberOfOpenCreditLinesAndLoans']
common_gmsc['grade']          = df_gmsc['NumberOfTimes90DaysLate']
common_gmsc['home_ownership'] = df_gmsc['NumberRealEstateLoansOrLines']
common_gmsc['purpose']        = df_gmsc['NumberOfTime30-59DaysPastDueNotWorse']
common_gmsc['zip_code']       = df_gmsc['NumberOfDependents'].fillna(0)
common_gmsc['loan_status']    = df_gmsc['loan_status']

for col in ['term','grade','home_ownership','purpose','zip_code']:
    common_gmsc[col] = LabelEncoder().fit_transform(common_gmsc[col].astype(str))
common_gmsc['annual_inc'] = StandardScaler().fit_transform(common_gmsc[['annual_inc']])

aif_gmsc = StandardDataset(
    common_gmsc,
    label_name='loan_status',
    favorable_classes=[1],
    protected_attribute_names=['zip_code'],
    privileged_classes=[[1]]
)

X_gmsc = aif_gmsc.features
y_gmsc = aif_gmsc.labels.ravel()

y_pred_gmsc = rf.predict(X_gmsc)
y_prob_gmsc = rf.predict_proba(X_gmsc)[:, 1]

pred_gmsc = aif_gmsc.copy()
pred_gmsc.labels = y_pred_gmsc.reshape(-1, 1)
pred_gmsc.scores = y_prob_gmsc.reshape(-1, 1)

roc_gmsc = RejectOptionClassification(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    low_class_thresh=0.01,
    high_class_thresh=0.99,
    num_class_thresh=100,
    num_ROC_margin=50,
    metric_name="Statistical parity difference",
    metric_ub=0.05,
    metric_lb=-0.05
).fit(aif_gmsc, pred_gmsc)

post_gmsc = roc_gmsc.predict(pred_gmsc)

pre_gmsc_metrics = perf_metrics(y_gmsc, y_pred_gmsc, y_prob_gmsc)
post_gmsc_metrics = perf_metrics(y_gmsc, post_gmsc.labels.ravel(), post_gmsc.scores.ravel())

metric_gmsc_pre = ClassificationMetric(aif_gmsc, pred_gmsc,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
metric_gmsc_post = ClassificationMetric(aif_gmsc, post_gmsc,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

fair_gmsc_pre = {
    'SPD': metric_gmsc_pre.statistical_parity_difference(),
    'DI': metric_gmsc_pre.disparate_impact(),
    'EOD': metric_gmsc_pre.equal_opportunity_difference(),
    'AOD': metric_gmsc_pre.average_odds_difference(),
    'BiasAmp': metric_gmsc_pre.between_group_generalized_entropy_index(),
    'Theil': metric_gmsc_pre.theil_index()
}
fair_gmsc_post = {
    'SPD': metric_gmsc_post.statistical_parity_difference(),
    'DI': metric_gmsc_post.disparate_impact(),
    'EOD': metric_gmsc_post.equal_opportunity_difference(),
    'AOD': metric_gmsc_post.average_odds_difference(),
    'BiasAmp': metric_gmsc_post.between_group_generalized_entropy_index(),
    'Theil': metric_gmsc_post.theil_index()
}

print("\n=== GiveMeSomeCredit (RF + ROC) ===")
print("Performance BEFORE ROC:", pre_gmsc_metrics)
print("Performance AFTER  ROC:", post_gmsc_metrics)
print("Fairness BEFORE ROC:", fair_gmsc_pre)
print("Fairness AFTER  ROC:", fair_gmsc_post)

# SHAP for GMSC
X_gmsc_df = pd.DataFrame(X_gmsc, columns=aif_gmsc.feature_names)
raw_shap_gmsc = explainer.shap_values(X_gmsc_df)
shap_gmsc = raw_shap_gmsc[1] if isinstance(raw_shap_gmsc, list) and len(raw_shap_gmsc) > 1 else raw_shap_gmsc

plt.figure(figsize=(10, 6))
shap.summary_plot(shap_gmsc, X_gmsc_df, show=False)
plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/exp9_gmsc_shap_global.png', dpi=150, bbox_inches='tight')
plt.close()



In [ ]:
x# -----------------------------
# Step 7: Save combined results to CSV
# -----------------------------
rows = [
    {
        'Dataset': 'LendingClub BEFORE ROC',
        **pre_metrics,
        **fairness_pre
    },
    {
        'Dataset': 'LendingClub AFTER ROC',
        **post_metrics,
        **fairness_post
    },
    {
        'Dataset': 'GermanCredit BEFORE ROC',
        **pre_gc_metrics,
        **fair_gc_pre
    },
    {
        'Dataset': 'GermanCredit AFTER ROC',
        **post_gc_metrics,
        **fair_gc_post
    },
    {
        'Dataset': 'GiveMeSomeCredit BEFORE ROC',
        **pre_gmsc_metrics,
        **fair_gmsc_pre
    },
    {
        'Dataset': 'GiveMeSomeCredit AFTER ROC',
        **post_gmsc_metrics,
        **fair_gmsc_post
    }
]

out_csv = f"{RESULTS_DIR}/exp9_rf_reject_option_validation_results.csv"
pd.DataFrame(rows).to_csv(out_csv, index=False)
print(f"\n✅ Results saved to: {out_csv}")
print(f"SHAP plots saved to: {RESULTS_DIR}/exp9_*_shap_*.png")